## EXPERIMENTO 1

* Métrica Perplejidad: https://en.wikipedia.org/wiki/Perplexity#cite_ref-1

* Loss: Categorical Crossentropy (buscar más referencias para ambos)

* Arquitectura: Capa average, capa max y capa densa

### Corpus

@InProceedings{Danescu-Niculescu-Mizil+Lee:11a,

  author={Cristian Danescu-Niculescu-Mizil and Lillian Lee},

  title={Chameleons in imagined conversations:

  A new approach to understanding coordination of linguistic style in dialogs.},

  booktitle={Proceedings of the

        Workshop on Cognitive Modeling and Computational Linguistics, ACL 2011},

  year={2011}

}

In [ ]:
## Falta el corpus y hacer train test split

In [14]:
from src import *

ModuleNotFoundError: No module named 'src'

In [9]:
from keras.callbacks import EarlyStopping
from keras.layers import average, Dense, maximum
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
import keras.utils as ku 
import numpy as np
from sklearn.base import BaseEstimator

In [10]:
class TextLSTM(BaseEstimator):
    
    def __init__(self, tokenizer=Tokenizer()):
        
        self.tokenizer = tokenizer
        
    def etl(self, data):

        # basic cleanup
        corpus = data.lower().split("\n")

        # tokenization
        self.tokenizer.fit_on_texts(corpus)
        self.total_words = len(self.tokenizer.word_index) + 1

        # create input sequences using list of tokens
        input_sequences = []
        for line in corpus:
            # TODO: Probar con fastText y HashingVectorizer y los demás de text de keras.
            # Onehot no hace falta si usamos embedding
            token_list = self.tokenizer.texts_to_sequences([line])[0]
            for i in range(1, len(token_list)):
                n_gram_sequence = token_list[: i + 1]
                input_sequences.append(n_gram_sequence)

        # pad sequences
        self.max_sequence_len = max([len(x) for x in input_sequences])
        input_sequences = np.array(
            pad_sequences(input_sequences, maxlen=self.max_sequence_len, padding="pre")
        )
        # create X and y
        X, y = input_sequences[:, :-1], input_sequences[:, -1]
        y = ku.to_categorical(y, num_classes=self.total_words)

        return X, y


    def fit(self, X, y, earlystop=False, epochs=200, batch_size=None, verbose=1, activation="softmax", optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]):

        self.net = Sequential()
        # Hace la media y el máximo respectivamente de dos capas (densas por ejemplo) Como usarlo?
        self.net.add(average())
        self.net.add(maximum())
        self.net.add(Dense(units=self.tokenizer.total_words, activation=activation))

        self.net.compile(
            loss=loss, optimizer=optimizer, metrics=metrics
        )
        if earlystop:
            earlystop = EarlyStopping(
                monitor="val_loss", min_delta=0, patience=5, verbose=0, mode="auto"
            )
            self.net.fit(X, y, epochs=epochs, batch_size=None, verbose=verbose, callbacks=[earlystop])
        else:
            self.net.fit(X, y, epochs=epochs, batch_size=None, verbose=verbose)
        print(self.net.summary())

        return self


    def generate_text(self, seed_text, next_words):
        
        for _ in range(next_words):
            token_list = self.tokenizer.texts_to_sequences([seed_text])[0]
            token_list = pad_sequences(
                [token_list], maxlen=self.max_sequence_len - 1, padding="pre"
            )
            predicted = self.net.predict_classes(token_list, verbose=0)

            output_word = ""
            for word, index in self.tokenizer.word_index.items():
                if index == predicted:
                    output_word = word
                    break
            seed_text += " " + output_word
        return seed_text

In [11]:
data = """Con diez cañones por banda,
viento en popa a toda vela,
no corta el mar, sino vuela,
un velero bergantín;
bajel pirata que llaman
por su bravura el Temido
en todo el mar conocido
del uno al otro confín.
La luna en el mar riela,
en la lona gime el viento
y alza en blando movimiento
olas de plata y azul;
y ve el capitán pirata,
cantando alegre en la popa,
Asia a un lado, al otro Europa,
Y allá a su frente Estambul:
-Navega, velero mío,
sin temor
que ni enemigo navío,
ni tormenta, ni bonanza
tu rumbo a torcer alcanza,
ni a sujetar tu valor.
Veinte presas
hemos hecho
a despecho
del inglés
y han rendido
sus pendones
cien naciones
a mis pies.
Que es mi barco mi tesoro,
que es mi Dios la libertad;
mi ley, la fuerza y el viento;
mi única patria, la mar.
Allá muevan feroz guerra
ciegos reyes
por un palmo más de tierra,
que yo tengo aquí por mío
cuanto abarca el mar bravío
a quien nadie impuso leyes.
Y no hay playa
sea cualquiera,
ni bandera
de esplendor,
que no sienta
mi derecho
y dé pecho
a mi valor
Que es mi barco mi tesoro,
que es mi Dios la libertad;
mi ley, la fuerza y el viento;
mi única patria, la mar.
A la voz de ¡barco viene!,
es de ver
cómo vira y se previene
a todo trapo a escapar:
que yo soy el rey del mar
y mi furia es de temer.
En las presas
yo divido
lo cogido
por igual:
sólo quiero
por riqueza
la belleza
sin rival.
Que es mi barco mi tesoro,
que es mi Dios la libertad;
mi ley, la fuerza y el viento;
mi única patria, la mar.
¡Sentenciado estoy a muerte!
Yo me río:
no me abandone la suerte,
y al mismo que me condena
colgaré de alguna antena
quizá en su propio navío.
Y si caigo,
¿qué es la vida?
Por perdida
ya la di
cuando el yugo
del esclavo
como un bravo sacudí.
Que es mi barco mi tesoro,
que es mi Dios la libertad;
mi ley, la fuerza y el viento;
mi única patria, la mar.
Son mi música mejor
aquilones,
el estrépito y temblor
de los cables sacudidos
del negro mar los bramidos
y el rugir de mis cañones.
Y del trueno
al son violento,
y del viento,
al rebramar,
yo me duermo
sosegado,
arrullado
por el mar.
Que es mi barco mi tesoro,
que es mi Dios la libertad;
mi ley, la fuerza y el viento;
mi única patria, la mar."""

In [12]:
model = TextLSTM()
X, y = model.etl(data)
model.fit(X, y, epochs=200)

TypeError: average() missing 1 required positional argument: 'inputs'